In [47]:
from dotenv import load_dotenv

load_dotenv()

True

In [1]:
import requests
import pandas as pd
import time

In [48]:
import os

In [49]:
# youtube api-key
API_KEY = os.getenv("API_KEY")

CHANNEL_ID = os.getenv("CHANNEL_ID") # we can change it later

In [8]:
# api calling

pageToken = ""
url = "https://www.googleapis.com/youtube/v3/search?key="+API_KEY+"&channelId="+CHANNEL_ID+"&part=snippet,id&order=date&maxResults=10000"+pageToken

response = requests.get(url).json()

Below are the details about first video. Like videoId, title, description, thumbnail(url height width), published time etc.

In [13]:
response['items'][0]

{'kind': 'youtube#searchResult',
 'etag': 'tJStzweYNGk6fOiOaoSlH2YcTZk',
 'id': {'kind': 'youtube#video', 'videoId': 'VEjxlKBkZGM'},
 'snippet': {'publishedAt': '2022-02-08T03:13:06Z',
  'channelId': 'UCW8Ews7tdKKkBT6GdtQaXvQ',
  'title': 'Amazon Data Science Interview Question Walkthrough | How to Deal with Streaks in SQL',
  'description': 'This walkthrough has been requested by multiple users on StrataScratch as it seems to have become quite a popular question.',
  'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/VEjxlKBkZGM/default.jpg',
    'width': 120,
    'height': 90},
   'medium': {'url': 'https://i.ytimg.com/vi/VEjxlKBkZGM/mqdefault.jpg',
    'width': 320,
    'height': 180},
   'high': {'url': 'https://i.ytimg.com/vi/VEjxlKBkZGM/hqdefault.jpg',
    'width': 480,
    'height': 360}},
  'channelTitle': 'StrataScratch',
  'liveBroadcastContent': 'none',
  'publishTime': '2022-02-08T03:13:06Z'}}

In [16]:
# grab the video ID

video_id = response['items'][0]['id']['videoId']
video_id

'VEjxlKBkZGM'

In [19]:
# grab the video Title

video_title = response['items'][0]['snippet']['title']
video_title

'Amazon Data Science Interview Question Walkthrough | How to Deal with Streaks in SQL'

In [20]:
video_title = str(video_title).replace("|", "")
video_title

'Amazon Data Science Interview Question Walkthrough  How to Deal with Streaks in SQL'

In [22]:
# grab the uploaded time

upload_date = response['items'][0]['snippet']['publishedAt']
upload_date

'2022-02-08T03:13:06Z'

Now we did all this for a single instance. But we need to grab all videos related data. So we will use for loop to grab every video. But we also need to grab statistics related to each video.

We can make another api call to get statistics related to a particular ID

In [24]:
# make another api call this time to get video statistics

url_video_stats = "https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&part=statistics&key="+API_KEY

response_video_stats = requests.get(url_video_stats).json()

In [33]:
response_video_stats

{'kind': 'youtube#videoListResponse',
 'etag': 'X3Nz3mDQEQT2WD0MKcvuGSYQ3-A',
 'items': [{'kind': 'youtube#video',
   'etag': 'gE7TfBNQxSFobzVFynjmh7bj5sY',
   'id': '5-1ClCuQCIw',
   'statistics': {'viewCount': '3171',
    'likeCount': '31',
    'favoriteCount': '0',
    'commentCount': '10'}}],
 'pageInfo': {'totalResults': 1, 'resultsPerPage': 1}}

In [35]:
view_count = response_video_stats['items'][0]['statistics']['viewCount']
like_count = response_video_stats['items'][0]['statistics']['likeCount']
favorite_count = response_video_stats['items'][0]['statistics']['favoriteCount']
comment_count = response_video_stats['items'][0]['statistics']['commentCount']

Since we are collecting statistics related to each video in variables. We can instead create a dataframe and store each variable values in columns. That would lead us to 7 columns in dataframe.

In [41]:
# calling this function to collect each video stat

def get_video_details(video_id):
    
    # collect video statistics
    url_video_stats = "https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&part=statistics&key="+API_KEY

    response_video_stats = requests.get(url_video_stats).json()
        
    view_count = response_video_stats['items'][0]['statistics']['viewCount']
    like_count = response_video_stats['items'][0]['statistics']['likeCount']
    favorite_count = response_video_stats['items'][0]['statistics']['favoriteCount']
    comment_count = response_video_stats['items'][0]['statistics']['commentCount']
    
    return view_count, like_count, favorite_count, comment_count

In [42]:
def get_videos(data):
    
    # make api call
    pageToken = ""
    url = "https://www.googleapis.com/youtube/v3/search?key="+API_KEY+"&channelId="+CHANNEL_ID+"&part=snippet,id&order=date&maxResults=10000"+pageToken
    response = requests.get(url).json()
    time.sleep(1)
    
    for video in response['items']:

        if video['id']['kind'] == 'youtube#video':

            video_id = video['id']['videoId']
            video_title = video['snippet']['title']
            upload_date = video['snippet']['publishedAt']

            view_count, like_count, favorite_count, comment_count = get_video_details(video_id)

            # append each video related data in a dataframe
            data = data.append(
                {
                    'video_id': video_id,
                    'video_title': video_title,
                    'upload_date': upload_date,
                    'view_count': view_count,
                    'like_count': like_count,
                    'favorite_count': favorite_count,
                    'comment_count': comment_count
                }, ignore_index=True
            )
    return data

In [39]:
data.head()

,video_id,video_title,upload_date,view_count,like_count,favorite_count,comment_count
0,VEjxlKBkZGM,Amazon Data Science Interview Question Walkthr...,2022-02-08T03:13:06Z,4376,160,0,16
1,hMUf7DqG1nQ,SQL Basics: How to Join Multiple Tables in SQL...,2022-01-27T06:23:05Z,2639,82,0,5
2,ToEl_tpyoM4,Uber Data Science Python Interview Question Wa...,2022-01-10T05:46:03Z,16789,433,0,23
3,30hS-MjpU6E,SQL Syntax Best Practices: How to Structure Yo...,2021-12-22T03:32:13Z,8307,386,0,42
4,Bgpp99iz0I0,Most Common Coding Mistakes on Data Science In...,2021-11-04T01:53:13Z,3910,189,0,30


In [43]:
# create empty dataframe

data = pd.DataFrame(columns=["video_id", "video_title", "upload_date", "view_count", "like_count", "favorite_count", "comment_count"])

data = get_videos(data)

In [45]:
data

,video_id,video_title,upload_date,view_count,like_count,favorite_count,comment_count
0,VEjxlKBkZGM,Amazon Data Science Interview Question Walkthr...,2022-02-08T03:13:06Z,4377,160,0,16
1,hMUf7DqG1nQ,SQL Basics: How to Join Multiple Tables in SQL...,2022-01-27T06:23:05Z,2639,82,0,5
2,ToEl_tpyoM4,Uber Data Science Python Interview Question Wa...,2022-01-10T05:46:03Z,16790,433,0,23
3,30hS-MjpU6E,SQL Syntax Best Practices: How to Structure Yo...,2021-12-22T03:32:13Z,8307,386,0,42
4,Bgpp99iz0I0,Most Common Coding Mistakes on Data Science In...,2021-11-04T01:53:13Z,3911,189,0,30
5,GeJUvdkJKEc,Multiple Solutions to Data Scientist Interview...,2021-10-11T06:50:13Z,6077,175,0,34
6,GGURenNfXI0,Multiple Approaches for Solving a Twitter Data...,2021-09-13T06:06:10Z,4217,174,0,32
7,PlpUo6bHsBQ,Advanced Facebook Data Science SQL interview q...,2021-08-09T02:18:01Z,15710,487,0,58
8,uY2wfR8Dkqo,5 Tips for a Successful Data Science Interview,2021-07-26T03:43:39Z,7985,228,0,40
9,yY7yau9j3xk,Data Science SQL Scenario Based Interview Ques...,2021-07-20T05:18:03Z,5705,155,0,27
